### Discrete regression

In [ ]:
import sys
sys.path.append('../../Utilities/src')
sys.path.append('../../Utilities')

import pystan
import stan_utility

import arviz as az
import numpy as np
import scipy.stats as stats

import pandas as pd


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.context('seaborn-white')
mpl.rcParams['figure.dpi']= 200

In [ ]:
from DA_tools.DA_tools import ribbon_plot
from DA_tools.DA_colors import *


In [ ]:
def integer_histogram_matrix(max_y,y_ppc):
    if len(y_ppc.shape)==1:
        y_ppc=np.expand_dims(y_ppc,axis=0)
    B=max_y+1
    bins = np.array([*range(B+1)])-0.5
    counts = [np.histogram(y_ppc[n], bins=bins)[0] for n in range(y_ppc.shape[0])]
    return bins, np.array(counts)

def pad_hist_for_plot(bins,counts):
    if len(counts.shape)==1:
        ax=0
    else: ax=1
        
    xs = (np.repeat(bins,repeats=2))[1:-1]
    pad_counts = np.repeat(counts,repeats=2,axis=ax)
    return xs, pad_counts

In [ ]:
df = pd.read_csv('discrete_regression_data.csv',index_col=0)

In [ ]:
df.describe()

### Prior predictive checks

In [ ]:
with open('poisson_ppc.stan', 'r') as file:
    print(file.read())
model_ppc = stan_utility.compile_model('poisson_ppc.stan')


In [ ]:
data_ppc = dict(M=3, N=1000,X=df.loc[:,'x_1':'x_3'].values,sigma=10)


R = 1000
sim_ppc=model_ppc.sampling(data=data_ppc, 
                           iter=R, warmup=0, 
                           chains=1, 
                           refresh=R,
                           algorithm='Fixed_param',
                           seed=29042020)

#### Exception warning
Setting too high $\sigma$ in priors results in following exceptions

```
Exception: poisson_log_rng: Log rate parameter[1] is 22.5551, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[13] is 22.6184, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[1] is 27.0051, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[16] is 25.1868, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[9] is 25.1681, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[8] is 26.3585, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[2] is 21.3792, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[13] is 23.3107, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[9] is 27.6798, but must be less than 20.7944  (in 'unknown file name' at line 17)

Exception: poisson_log_rng: Log rate parameter[9] is 22.5301, but must be less than 20.7944  (in 'unknown file name' at line 17)
```

#### What is going on?

```poisson_log_rng ``` and consequently ```poison_log``` are operating as Poisson distribution in the form
$$
y\sim\mathrm{Poisson}(\exp(\theta))
$$
where $\theta$ is the parametr of interest. And what is the interpretation of this parameter value of ```20.7944```? It is the rate of Poisson distribution:


In [ ]:
lam = np.exp(20.7944)
print(lam)

But this is just a number, however lets take its base 2 logarithm

In [ ]:
np.log2(lam)

This is almost 30, and $2^{30}$ is a limit of ```int32``` capacity. 

#### Prior tuning
Setting $\sigma=10$ would be fine on linear scale, but in GLMs we generally need to be careful. 
If we choose $\sigma=2$ having predictors bounded to $[-1,1]$ it still has a possibility to cover relatively very large numbers if needed, but without warnings. 


Our data was limited to 28, but to stay on the safe side let us limit our priors to keep us under few thousand. For $\sigma=2$ there is around 1% probability (assuming all predictors at maximum), that 
$$ X\beta+\alpha > 8 $$

And this corresponds to $\lambda$

In [ ]:
lam_ub=np.exp(8)
print('Lambda upper bound: {0:4.2f}'.format(lam_ub))

Using familiar formula, we have $y_\mathrm{bound}$ of  to be 
$$ \lambda +3\sqrt{\lambda}=y_\mathrm{bound}$$

In [ ]:
y_bound=np.int(lam_ub+3*np.sqrt(lam_ub))
print('Upper value of y:{}'.format(y_bound))




### Prior predictions

In [ ]:
data_ppc = dict(M=3, N=1000,X=df.loc[:,'x_1':'x_3'].values,sigma=2)


R = 1000
sim_ppc=model_ppc.sampling(data=data_ppc, 
                           iter=R, warmup=0, 
                           chains=1, 
                           refresh=R,
                           algorithm='Fixed_param',
                           seed=29042020)

params = sim_ppc.extract()
pars_mat=np.concatenate((params['beta'],np.expand_dims(params['alpha'],axis=1)),axis=1)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(7, 8), squeeze=False,sharex=True)
axes_flat=axes.flatten()
names_of_pars = [r'$\beta_1$',r'$\beta_2$',r'$\beta_3$',r'$\alpha$',r'$\phi$',r'$\psi=\phi^{-1}$']
for k in range(len(axes_flat)):
    ax = axes_flat[k]
    ax.hist(pars_mat[:,k],bins=20,color=DARK,edgecolor=DARK_HIGHLIGHT,density=True)
    ax.set_title(names_of_pars[k])
    ax.set_yticks([])
fig.tight_layout()

plt.show()

In [ ]:
np.max(params['lambda'][0])

In [ ]:
y_ppc = (params['y_ppc'])
max_y = 50

bins,counts = integer_histogram_matrix(max_y,y_ppc)
xs, pad_counts = pad_hist_for_plot(bins,counts)    
obs_counts = np.histogram(df['y'], bins=bins)[0]
_, pad_obs_counts = pad_hist_for_plot(bins,obs_counts)


fig, ax = plt.subplots(1, 1, figsize=(7,4))

ax=ribbon_plot(xs,pad_counts,ax)

ax.plot(xs, pad_obs_counts, linewidth=2.5, color="white",zorder=1)
ax.plot(xs, pad_obs_counts, linewidth=2.0, color="black",zorder=2)
ax.set_xlim([min(bins), max(bins)])
ax.set_xlabel("y")
ax.set_ylim([0, max(max(obs_counts), np.max(counts))])
ax.set_ylabel("Prior Predictive Distribution")

plt.show()

### Posterior inference and prediction

In [ ]:
model = stan_utility.compile_model('poisson2.stan')
with open('poisson2.stan', 'r') as file:
    print(file.read())

In [ ]:
data = dict(M=3, N=1000,X=df.loc[:,'x_1':'x_3'].values,y=df.y.values)

fit = model.sampling(data=data, seed=12062020)

In [ ]:
params_1 = fit.extract()
pars_mat_1=np.concatenate((params_1['beta'],np.expand_dims(params_1['alpha'],axis=1)),axis=1)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(7, 8), squeeze=False)
axes_flat=axes.flatten()
for k in range(len(axes_flat)):
    ax = axes_flat[k]
    ax.hist(pars_mat_1[:,k],bins=20,color=DARK,edgecolor=DARK_HIGHLIGHT,density=True)
    ax.set_title(names_of_pars[k])
    ax.set_yticks([])
fig.tight_layout()

plt.show()

In [ ]:
max_y = 50

y_prediction_1 = (params_1['y_ppc'])

bins,counts = integer_histogram_matrix(max_y,y_prediction_1)
xs, pad_counts_pred = pad_hist_for_plot(bins,counts)    
#obs_counts = np.histogram(df['y'], bins=bins)[0]
#_, pad_obs_counts = pad_hist_for_plot(bins,obs_counts)


fig, ax = plt.subplots(1, 1, figsize=(7,4))

ax=ribbon_plot(xs,pad_counts_pred,ax)

ax.plot(xs, pad_obs_counts, linewidth=2.5, color="white",zorder=1)
ax.plot(xs, pad_obs_counts, linewidth=2.0, color="black",zorder=2)

ax.set_xlim([min(bins), max(bins)])
ax.set_xlabel("y")
ax.set_ylim([0, max(max(obs_counts), np.max(counts))])
ax.set_ylabel("Posterior Predictive Distribution")
ax.set_title('Using Poisson model leads to systematic differences')


plt.show()

### Modelling dispersion in discrete regrssion

Negative binomial distribution in dispersion parametrization is equivalnent to concentration parametrization but with concentration parameter $\phi$ replaced with dispersion $\psi$,
$$
\psi = \frac{1}{\phi}.
$$

|Parameter Name|Symbol|Domain|Units|
|--- |--- |--- |--- |
|Intensity|$$\mu$$|$$\mathbb{R}^{+}$$|$$[x]$$|
|Dispersion|$$\psi$$|$$\mathbb{R}^{+}$$|$$[x^{-1}]$$|

This leads to following formulae:

|||
|--- |--- |
|Space|$$X = \mathbb{N}$$|
|Density|$$\pi(x; \mu, \psi) = {x + \psi^{-1} - 1 \choose x}
          \left( \frac{\mu \cdot \psi }{\mu \cdot \psi + 1} \right)^{x} \
          \left( \frac{1}{\mu \cdot \psi + 1} \right)^{\frac{1}{\psi}}$$|
|Mean|$$\mu$$|
|Variance|$$\mu + \mu^{2} \cdot \psi$$|

This is advantageous, as when $\psi$ tends to zero, then Negaitve Binomial distribution tends to Poisson distribution.

For more details [see](https://betanalpha.github.io/assets/case_studies/probability_densities.html#36_the_negative_binomial_family_2:_electric_boogaloo).

In [ ]:
model_nb = stan_utility.compile_model('negative_binomial.stan')
with open('negative_binomial.stan', 'r') as file:
    print(file.read())

In [ ]:
fit = model_nb.sampling(data=data, seed=4938483)

# Check diagnostics
stan_utility.check_all_diagnostics(fit)

params_nb = fit.extract()


In [ ]:
pars_mat_nb=np.concatenate((params_nb['beta'],
                         np.expand_dims(params_nb['alpha'],axis=1), 
                         np.expand_dims(params_nb['phi'],axis=1),
                         np.expand_dims(params_nb['inv_phi'],axis=1)),
                        axis=1)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(7, 8), squeeze=False)
axes_flat=axes.flatten()
for k in range(len(axes_flat)):
    ax = axes_flat[k]
    ax.hist(pars_mat_nb[:,k],bins=20,color=DARK,edgecolor=DARK_HIGHLIGHT,density=True)
    ax.set_title(names_of_pars[k])
#    ax.plot([tv[k],tv[k]],[0,5],linestyle='--',color='black')
    #ax.set_xticks([0,1,2,3,4,5,6])
    ax.set_yticks([])
fig.tight_layout()

plt.show()

In [ ]:
max_y = 50

y_prediction_2 = (params_nb['y_ppc'])

bins,counts = integer_histogram_matrix(max_y,y_prediction_2)
xs, pad_counts_pred_2 = pad_hist_for_plot(bins,counts)    
#obs_counts = np.histogram(df['y'], bins=bins)[0]
#_, pad_obs_counts = pad_hist_for_plot(bins,obs_counts)


fig, ax = plt.subplots(1, 1, figsize=(7,4))

ax=ribbon_plot(xs,pad_counts_pred_2,ax)

ax.plot(xs, pad_obs_counts, linewidth=2.5, color="white",zorder=1)
ax.plot(xs, pad_obs_counts, linewidth=2.0, color="black",zorder=2)

ax.set_xlim([min(bins), max(bins)])
ax.set_xlabel("y")
ax.set_ylim([0, max(max(obs_counts), np.max(counts))])
ax.set_ylabel("Posterior Predictive Distribution")
m_psi=np.mean(params_nb['inv_phi'])
sd_psi=np.std(params_nb['inv_phi'])
ax.set_title('NB distribution, with dispersion {0:1.3f} $\pm$ {1:1.3f} fits data better'.format(m_psi,sd_psi))
plt.show()

